In [1]:
import sys
from datetime import date
from pathlib import Path
import pickle
from typing import List, Tuple

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import Input, Model

sys.path.append("..")
from btc_predictor.config import config, logging_config
from btc_predictor.datasets import BitfinexCandlesAPIData
from btc_predictor.models import LSTMModelHandler

# from btc_predictor.datasets import DataReader
# from btc_predictor.models import LSTM_Model
# from btc_predictor.utils import print_metrics
# from btc_predictor.utils import show_plot
# from btc_predictor.utils import plot_train_history

tf.random.set_seed(78)
np.random.seed(78)
mpl.rcParams['figure.figsize'] = [16.0, 9.0]
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
candles = BitfinexCandlesAPIData()
candles.load(start_time=1610000000000)
df = candles.pd

In [3]:
BTCUSD = df.copy()

BTCUSD['baseline_close'] = BTCUSD['close'].shift(1)
BTCUSD['log_ret'] = np.log(BTCUSD['close']) - np.log(BTCUSD['close'].shift(1))
BTCUSD['baseline_log_ret'] = np.log(BTCUSD['baseline_close']) - np.log(BTCUSD['baseline_close'].shift(1))
BTCUSD.dropna(inplace=True)
print(f'Total daily data: {BTCUSD.shape[0]} days')

Total daily data: 9998 days


In [4]:
x = np.arange(1, 16)
x_log_ret = np.diff(np.log(x))
x_log_ret_diff = np.diff(x_log_ret)

In [5]:
x, x_log_ret, x_log_ret_diff

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
 array([0.69314718, 0.40546511, 0.28768207, 0.22314355, 0.18232156,
        0.15415068, 0.13353139, 0.11778304, 0.10536052, 0.09531018,
        0.08701138, 0.08004271, 0.07410797, 0.06899287]),
 array([-0.28768207, -0.11778304, -0.06453852, -0.04082199, -0.02817088,
        -0.02061929, -0.01574836, -0.01242252, -0.01005034, -0.0082988 ,
        -0.00696867, -0.00593474, -0.0051151 ]))

In [6]:
pred = -0.006

In [7]:
x_log_ret[-1] + pred

0.06299287148695165

In [8]:
x[-1] * np.exp(x_log_ret[-1] + pred)

15.975288708009678